# **Eat Well, Feel Rad !!**

## Installing Gurobipy

In [ ]:
!pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 23.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd

## Let's take a look at the Data we are working on

In [ ]:
food_data = pd.read_excel('/content/data.xlsx', sheet_name='Sheet1')
male_data = pd.read_excel('/content/data.xlsx', sheet_name='Sheet2')
female_data = pd.read_excel('/content/data.xlsx', sheet_name='Sheet3')
female_data=female_data.drop(['Unnamed: 0','Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'], axis=1)

In [ ]:
food_data.head()

,Food,Protein,Calories,Carbohydrates,Fat,Fiber,Cost,Integer
0,Food_Item_1,0.93,380.67,90.60,0.00,2.23,3.07,Y
1,Food_Item_2,8.84,692.00,13.19,71.97,9.60,7.07,Y
2,Food_Item_3,1.98,24.33,5.88,0.85,2.33,13.97,N
3,Food_Item_4,12.63,366.33,74.13,4.38,7.33,5.90,Y
4,Food_Item_5,0.43,144.00,30.07,1.67,0.97,1.07,N


In [ ]:
male_data.head()

,Age Lower,Age Upper,Protein Lower,Protein Upper,Calories Lower,Calories Upper,Carbohydrates Lower,Carbohydrates Upper,Fat Lower,Fat Upper,Fiber Lower,Fiber Upper
0,2,3,13,19,1000,1400,130,200,30,40,14,20
1,4,8,16,28,1200,1800,130,300,25,35,19,25
2,9,13,27,46,1600,2200,130,400,25,35,25,31
3,14,18,52,82,1800,2800,130,500,25,35,30,38
4,19,30,56,82,2000,3000,130,550,20,35,38,38


In [ ]:
female_data.head()

,Age Lower,Age Upper,Protein Lower,Protein Upper,Calories Lower,Calories Upper,Carbohydrates Lower,Carbohydrates Upper,Fat Lower,Fat Upper,Fiber Lower,Fiber Upper
0,2,6,11,22,1000,1300,130,200,30,50,14,23
1,7,9,20,56,1000,2000,130,280,30,85,11,26
2,10,12,28,62,1400,2400,130,273,39,71,19,26
3,15,18,46,71,1800,2400,130,260,46,71,22,28
4,19,24,46,71,1800,2200,130,260,46,71,22,28


## Gurobipy Optimizer

In [ ]:
import gurobipy as gp
import pandas as pd

# Read data from Excel sheets
food_data = pd.read_excel('/content/data.xlsx', sheet_name='Sheet1')
male_data = pd.read_excel('/content/data.xlsx', sheet_name='Sheet2')
female_data = pd.read_excel('/content/data.xlsx', sheet_name='Sheet3')
female_data=female_data.drop(['Unnamed: 0','Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'], axis=1)

# User Input
age = int(input('Enter your age: '))
gender = input('Enter your gender (M/F): ')
budget_up = float(input("Enter your budget's upper limit: "))
budget_low = float(input("Enter your budget's lower limit: "))

if budget_low > budget_up:
    print("Re-enter the budget limits.")
    exit(1)

# Create optimization model
model = gp.Model('Nutrition Optimization')

# Deciding the type of variable, INTEGER or NON-INTEGER
types = []
for i in range(len(food_data)):
    if food_data.iloc[i, 7] == 'Y':
        types.append(gp.GRB.INTEGER)
    else:
        types.append(gp.GRB.CONTINUOUS)

# Define decision x for each food item
foods = list(food_data['Food'])
x = model.addVars(foods, lb=0, ub=gp.GRB.INFINITY, vtype=types, name='servings')

# Define the budget constraints
costs = dict(zip(foods, food_data['Cost']))
model.addConstr(gp.quicksum(costs[i] * x[i] for i in foods) <= budget_up)
model.addConstr(gp.quicksum(costs[i] * x[i] for i in foods) >= budget_low)

# Find correct row from male_data or female_data based on age and gender
if gender == 'M':
    age_group_data = male_data[(male_data['Age Lower'] <= age) & (male_data['Age Upper'] >= age)]
elif gender == 'F':
    age_group_data = female_data[(female_data['Age Lower'] <= age) & (female_data['Age Upper'] >= age)]
else:
    print(f"Please use either M or F as input for gender.")
    exit(1)

# Nutrient Constraints
nutrients = ['Protein', 'Calories', 'Carbohydrates', 'Fat', 'Fiber']
for nutr in nutrients:
    lb = age_group_data[nutr + ' Lower'].values[0]
    ub = age_group_data[nutr + ' Upper'].values[0]
    amounts = dict(zip(foods, food_data[nutr]))
    model.addConstr(gp.quicksum(amounts[i] * x[i] for i in foods) >= lb)
    model.addConstr(gp.quicksum(amounts[i] * x[i] for i in foods) <= ub)

# Saving the model
model.write("/content/model.lp")
# Silencing the output
model.setParam("OutputFlag", False)

# Add solution pool
model.params.PoolSearchMode = 2 # exhaustively search for good solutions
model.params.PoolGap = 0.1 # set optimality gap tolerance to 50%
model.params.PoolSolutions = 10 # set maximum number of solutions to store in pool to 10
model.params.MIPFocus = 2 # emphasize it to find good feasible solutions

# solve optimization model
model.optimize()

# print optimal solution and other good solutions (if any)
if model.status == gp.GRB.OPTIMAL:
    # print solutions from the solution pool
    for j in range(model.SolCount):
        if j == 0:
            print('\nOptimal Solution')
        else:
            print(f'\nOther Solution #{j+1}')

        model.params.SolutionNumber = j
        total_cost = 0
        for i in foods:
            if x[i].Xn > 0.1:
                print(f'{i}: {x[i].Xn:.2f} servings')
                total_cost += x[i].Xn*costs[i]
        print(f'Total Cost: ${total_cost:.2f}\n')

else:
    print('No feasible solution found.')

Enter your age: 10
Enter your gender (M/F): M
Enter your budget's upper limit: 1000
Enter your budget's lower limit: 500
Restricted license - for non-production use only - expires 2025-11-24

Optimal Solution
Food_Item_13: 21.25 servings
Food_Item_19: 1.33 servings
Food_Item_49: 0.29 servings
Food_Item_89: 7.41 servings
Food_Item_95: 0.84 servings
Total Cost: $500.00


Other Solution #2
Food_Item_13: 24.80 servings
Food_Item_50: 4.00 servings
Food_Item_94: 3.00 servings
Food_Item_99: 1.00 servings
Total Cost: $562.35


Other Solution #3
Food_Item_13: 25.65 servings
Food_Item_50: 5.00 servings
Food_Item_59: 1.02 servings
Food_Item_94: 2.00 servings
Food_Item_95: 0.69 servings
Total Cost: $498.44


Other Solution #4
Food_Item_13: 24.80 servings
Food_Item_50: 4.00 servings
Food_Item_59: 0.79 servings
Food_Item_94: 3.00 servings
Total Cost: $549.78


Other Solution #5
Food_Item_13: 24.75 servings
Food_Item_50: 4.00 servings
Food_Item_94: 3.00 servings
Food_Item_98: 1.00 servings
Total Cost

## Genetic Algorithm

In [ ]:
import pandas as pd
import random
import numpy as np

# Data Loading
food_data = pd.read_excel('/content/data.xlsx', sheet_name='Sheet1')
male_data = pd.read_excel('/content/data.xlsx', sheet_name='Sheet2')
female_data = pd.read_excel('/content/data.xlsx', sheet_name='Sheet3')
female_data=female_data.drop(['Unnamed: 0','Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'], axis=1)

age =int(input('Enter your age: '))
gender = input('Enter your gender (M/F): ')
budget_low = float(input("Enter your budget's lower limit: "))
budget_up = float(input("Enter your budget's upper limit: "))

if gender == 'M':
    age_group_data = male_data[(male_data['Age Lower'] <= age) & (male_data['Age Upper'] >= age)]
elif gender == 'F':
    age_group_data = female_data[(female_data['Age Lower'] <= age) & (female_data['Age Upper'] >= age)]

# Helper Functions
def generate_meal_plan(foods):
    meal_plan = {}
    for food in random.sample(foods, k=10):
        meal_plan[food] = random.uniform(0.5, 2)
    return meal_plan

def calculate_nutrients(meal_plan):
    # Nutrient calculation logic
    nutrient_values = {}
    for food, servings in meal_plan.items():
        for nutrient in ['Protein', 'Calories', 'Carbohydrates', 'Fat', 'Fiber']:
            nutrient_values.setdefault(nutrient, 0)
            nutrient_values[nutrient] += servings * food_data.loc[food_data['Food'] == food, nutrient].iloc[0]
    return nutrient_values

def calculate_fitness(meal_plan, constraints, age_group_data):
    total_cost = sum(meal_plan[food] * food_data.loc[food_data['Food'] == food, 'Cost'].iloc[0] for food in meal_plan)
    nutrient_values = calculate_nutrients(meal_plan)

    penalty = 0

    # Scaled Budget Penalty
    if total_cost < constraints['budget_low'] or total_cost > constraints['budget_up']:
        budget_deviation = abs(total_cost - constraints['budget_up'])
        budget_penalty = budget_deviation / constraints['budget_up']  # Scale by budget limit
        penalty += budget_penalty

    # Nutritional Penalties
    for nutr in ['Protein', 'Calories', 'Carbohydrates', 'Fat', 'Fiber']:
        lb = constraints[nutr + '_Lower']
        ub = constraints[nutr + '_Upper']
        if nutrient_values[nutr] < lb or nutrient_values[nutr] > ub:
            penalty += abs(nutrient_values[nutr] - lb)

    fitness_score = 1 / (penalty + 1)
    return fitness_score

# Evolutionary Algorithm
population_size = 50
num_generations = 50
foods = list(food_data['Food'])

# Constraints
constraints = {
    'budget_low': budget_low,
    'budget_up': budget_up,
    'Protein_Lower': age_group_data['Protein Lower'].values[0],
    'Protein_Upper': age_group_data['Protein Upper'].values[0],
    'Calories_Lower': age_group_data['Calories Lower'].values[0],
    'Calories_Upper': age_group_data['Calories Upper'].values[0],
    'Carbohydrates_Lower': age_group_data['Carbohydrates Lower'].values[0],
    'Carbohydrates_Upper': age_group_data['Carbohydrates Upper'].values[0],
    'Fat_Lower': age_group_data['Fat Lower'].values[0],
    'Fat_Upper': age_group_data['Fat Upper'].values[0],
    'Fiber_Lower': age_group_data['Fiber Lower'].values[0],
    'Fiber_Upper': age_group_data['Fiber Upper'].values[0]
}


# Initialization
population = [generate_meal_plan(foods) for _ in range(population_size)]

# Evolution
for _ in range(num_generations):
    fitness_scores = [calculate_fitness(plan, constraints, age_group_data) for plan in population]

    # Selection (Roulette Wheel)
    total_fitness = sum(fitness_scores)
    probabilities = [score / total_fitness for score in fitness_scores]
    parents = np.random.choice(population, size=population_size, replace=True, p=probabilities)

    # Crossover (Single point)
    offspring = []
    for i in range(0, population_size, 2):
        split_point = random.randint(1, len(parents[i]) - 1)
        offspring_1 = {}
        offspring_2 = {}

        items_list = list(parents[i].items())
        for food, servings in items_list[:split_point]:
          offspring_1[food] = servings
          offspring_2[food] = servings

        for food, servings in items_list[split_point:]:
          offspring_1[food] = servings

        for food, servings in items_list[split_point:]:
          offspring_2[food] = servings

        offspring.append(offspring_1)
        offspring.append(offspring_2)

    # Mutation
    for plan in offspring:
        if random.random() < 0.2:
            food_items = list(plan.keys())
            i, j = random.sample(range(len(food_items)), k=2)
            food1, food2 = food_items[i], food_items[j]
            plan[food1], plan[food2] = plan[food2], plan[food1]


    population = offspring

def display_results(population, food_data):
    top_results = sorted(population, key=lambda plan: calculate_fitness(plan, constraints, age_group_data), reverse=True)[:5]

    for i, meal_plan in enumerate(top_results):
        print(f"\nSolution #{i+1}")
        total_cost = 0
        for food, servings in meal_plan.items():
            cost = food_data.loc[food_data['Food'] == food, 'Cost'].iloc[0]
            print(f"{food}: {servings:.2f} servings (${servings * cost:.2f})")
            total_cost += servings * cost
        print(f"Total Cost: ${total_cost:.2f}")
        print(f"Nutritional Breakdown: {calculate_nutrients(meal_plan)}")

display_results(population, food_data)


Enter your age: 10
Enter your gender (M/F): M
Enter your budget's lower limit: 500
Enter your budget's upper limit: 1000

Solution #1
Food_Item_54: 1.01 servings ($5.30)
Food_Item_64: 1.33 servings ($19.98)
Food_Item_67: 0.86 servings ($22.83)
Food_Item_18: 1.80 servings ($135.06)
Food_Item_74: 0.60 servings ($11.56)
Food_Item_39: 1.11 servings ($15.90)
Food_Item_11: 1.56 servings ($10.64)
Food_Item_97: 1.84 servings ($16.26)
Food_Item_59: 0.78 servings ($6.53)
Food_Item_69: 0.65 servings ($73.97)
Total Cost: $318.04
Nutritional Breakdown: {'Protein': 86.13228238694275, 'Calories': 1708.3089992384853, 'Carbohydrates': 290.9934207896397, 'Fat': 30.049313814404805, 'Fiber': 39.00667394341236}

Solution #2
Food_Item_54: 1.01 servings ($5.30)
Food_Item_64: 1.33 servings ($19.98)
Food_Item_67: 0.86 servings ($22.83)
Food_Item_18: 1.80 servings ($135.06)
Food_Item_74: 0.60 servings ($11.56)
Food_Item_39: 1.11 servings ($15.90)
Food_Item_11: 1.56 servings ($10.64)
Food_Item_97: 1.84 servings 